In [1]:
import pandas as pd

In [2]:
from sklearn.datasets import make_blobs

In [3]:
X, y, c = make_blobs(n_samples=3000, n_features=3, centers=5, cluster_std=1.0, center_box=(-150.0, 15.0), shuffle=True, random_state=0, return_centers=True)

In [4]:
data = pd.DataFrame(X, columns=[f'x{i+1}' for i in range(X.shape[1])])

In [5]:
data.to_parquet('sample.parquet')

# Case 1

In [6]:
# Создание конфигурации
config = {
    "data_source": "pandas",
    "data_path": "sample.parquet",
    "algorithm": "kmeans",
    "params": {
        "n_clusters": [3, 5],
        "init": ["k-means++"],
        'max_iter': [100]
    },
    "metric": "silhouette",
    "output_path": "best_kmeans.joblib"
}

# config = {
#   "data_source": "spark",
#   "data_path": "hdfs://data",
#   "preprocessing": {
#     "normalization": {
#       "methods": {
#         "feature1": "minmax",
#         "feature2": "zscore"
#       }
#     }
#   },
#   "algorithm": "kmeans",
#   "params": {"n_clusters": [3,5]},
#   "metric": "silhouette"
# }


# Сохранение конфига
import json
with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [7]:
# Инициализация компонентов
from data.loaders import PandasDataLoader
from models.sklearn_cluster import SklearnKMeans
import pandas as pd
from metrics.quality import SilhouetteScore
from optimization.strategies import GridSearch

data = pd.read_parquet(config['data_path'])
loader = PandasDataLoader(data)

# Ручной запуск пайплайна
optimizer = GridSearch()
best_params = optimizer.find_best(
    SklearnKMeans,
    loader,
    config['params'],
    SilhouetteScore()
)

# Обучение финальной модели
best_model = SklearnKMeans(best_params)
best_model.fit(loader)
best_model.save(config['output_path'])

In [8]:
best_params

{'n_clusters': 5, 'init': 'k-means++', 'max_iter': 100}

In [9]:
# Загрузка модели
model = SklearnKMeans.load("best_kmeans.joblib")

# Новые данные для предсказания
new_data = pd.DataFrame({
    'x1': [0.5, 1.5, 2.5],
    'x2': [0.3, 1.2, 2.1],
    'x3': [0.7, 3.5, 6.1],
})

# Предсказание кластеров
predictions = model.predict(new_data)
print(f"Предсказанные кластеры: {predictions.values}")

# Доступ к центроидам
print("Центроиды кластеров:")
for i, centroid in enumerate(model.model_data['centroids']):
    print(f"Кластер {i}: {centroid}")

Предсказанные кластеры: [2 2 2]
Центроиды кластеров:
Кластер 0: [-86.71765646 -19.37661801 -62.7420693 ]
Кластер 1: [ -56.34177069    2.69520837 -138.27478386]
Кластер 2: [-77.79753144  -2.81724278   9.02471286]
Кластер 3: [-60.16579643 -80.10752269 -43.40913948]
Кластер 4: [-59.48945757 -32.06058728 -50.50337955]


# Case 2

In [10]:
import pandas as pd
from core.api import train_pipeline
from config.registries import METRIC_REGISTRY, MODEL_REGISTRY
from core.factory import factory

In [11]:
# 1. Выбор алгоритма
print("Доступные алгоритмы:", list(MODEL_REGISTRY.keys()))

Доступные алгоритмы: ['kmeans', 'dbscan']


In [12]:
# 2. Просмотр параметров алгоритма
kmeans_params = MODEL_REGISTRY['kmeans']['params_help']
print("Параметры KMeans:")
for param, desc in kmeans_params.items():
    print(f"  {param}: {desc}")

Параметры KMeans:
  n_clusters: Number of clusters (positive integer)
  init: Initialization method [k-means++, random]
  max_iter: Maximum iterations (positive integer)


In [13]:
df = pd.read_parquet("sample.parquet")

# Обучение модели
model = train_pipeline(
    data_src=df,
    algorithm="kmeans",
    param_grid={
        "n_clusters": [3, 4, 5, 6 , 7],
        "init": ["k-means++", "random"],
        'max_iter': [100]},
    metric="silhouette",
    optimizer="random"
)
# Сохранение и загрузка
model.save("best_model.joblib")
# Анализ результатов
print(f"Best params: {model.model.get_params()}")
print(f"Инерция: {model.model.inertia_}")
print(f"Cluster centroids: {model.model_data['centroids']}")

Best params: {'algorithm': 'lloyd', 'copy_x': True, 'init': 'random', 'max_iter': 100, 'n_clusters': 5, 'n_init': 'auto', 'random_state': None, 'tol': 0.0001, 'verbose': 0}
Инерция: 8989.511303524969
Cluster centroids: [[ -59.48945757  -32.06058728  -50.50337955]
 [ -77.79753144   -2.81724278    9.02471286]
 [ -56.34177069    2.69520837 -138.27478386]
 [ -60.16579643  -80.10752269  -43.40913948]
 [ -86.71765646  -19.37661801  -62.7420693 ]]


In [14]:
model = MODEL_REGISTRY['kmeans'].get('class').load("best_model.joblib")

# Новые данные для предсказания
new_data = pd.DataFrame({
    'x1': [0.5, 1.5, 2.5],
    'x2': [0.3, 1.2, 2.1],
    'x3': [0.7, 3.5, 6.1],
})

# Предсказание кластеров
predictions = model.predict(new_data)
print(f"Предсказанные кластеры: {predictions.values}")

# Доступ к центроидам
print("Центроиды кластеров:")
for i, centroid in enumerate(model.model_data['centroids']):
    print(f"Кластер {i}: {centroid}")

Предсказанные кластеры: [1 1 1]
Центроиды кластеров:
Кластер 0: [-59.48945757 -32.06058728 -50.50337955]
Кластер 1: [-77.79753144  -2.81724278   9.02471286]
Кластер 2: [ -56.34177069    2.69520837 -138.27478386]
Кластер 3: [-60.16579643 -80.10752269 -43.40913948]
Кластер 4: [-86.71765646 -19.37661801 -62.7420693 ]
